TODO  removeLinks, setLinks
атрибуты для формального понятия: Destructor, 
метод для формального понятия remove_object(g)

Данная работа была сделана с помощью библиоеки fcapy (автор: Егор Дудырев)
ссылка на репозиторий: https://github.com/EgorDudyrev

In [ ]:
!pip install -U -q fcapy==0.1.3

In [322]:
global g
global L

### Функция isOld(c,g) проверяет, является ли понятие старым

In [323]:
def isOld(c, g):
    
    if g in L[c].extent:
        return False
    else:
        return True

### Процедура ConfirmDeletion(c) проверяет, является ли формальное понятия "c" удаленным или нет

In [324]:
# на входе в процедуру объект g уже удален из L
def ConfirmDeletion(c):
    oldChildren = 0
    # находим старое понятие среди нижних сосдедей c и определяем деструктор
    for child in L.subconcepts_dict[c]:
        if isOld(child, g) == True:
            if oldChildren == 0:
                destructor = child
                oldChildren = 1
            else:
                oldChildren = 2
                break
                
    # проверка на однозначное соответствие между деструктором и удаленным понятием
    if oldChildren == 1 and destructor.Extent == L[c].Extent:
        L[c].Destructor = destructor  
        return True
    else:
        return False

### Процедура FastDeletion(L, c) необходим для фиксирования отношения покрытия

In [325]:
# фиксация отношения покрытия
def FastDeletion(L, c):
    
    if ConfirmDeletion(c):  # проверка на удаленное понятие
        L[c].IsDeleted = True  # помечается удаленным
        removeLinks(L[c], L[c].Destructor)  # Удаляем связи между понятием c и его деструктором
        L.remove_concept(c)  # Удаляем понятие c из решетки L
        
        for parent in L.superconcepts_dict[c]:
            if parent.IsDeleted or parent.IsModified:
                continue
            else:
                FastDeletion(L, parent)
                
        for parent in L.superconcepts_dict[c]:
            if parent.IsModified:  # для изменения отношения покрытия проверяем только измененные понятия
                removeLinks(parent, L[c])  # Для каждого верхнего соседа c удаляем связи с  понятием c
                covered = False
                
                for child in L.subconcepts_dict[parent]:  # проверяем нужны ли связи
                    if isOld(child, g) and child.intent.__le__(L[c].Destructor.intent):  # __le__ - является подмножеством
                        covered = True
                        break
                        
                if covered == False:
                    setLinks(parent, c.Destructor)  # Set links between parent and c.Destructor
                
    else:
        L[c].IsModified = True  # помечается как измененное
        for parent in L.superconcepts_dict[c]:  # все более общие являются измененными
            parent.IsModified = True

### Процедура DeleteInstance(L, g) удаляет объект g из решетки L и изменяет понятия формальные понятия решетки.

In [341]:
global affectedConcepts
def DeleteInstance(L, g):
    minConcept = L[max(L.trace_context(K)[-1][g])]  # = ({g}↑↓, {g}↑) менее общее понятие, которое содержит удаляемый объект g
    affectedConcepts.append(minConcept)  # Добавляем minConcept в список посещенных понятий affectedConcepts
    minConcept.Affected = True  # Помечаем minConcept как посещенное понятие
    while affectedConcepts != 0:
        c = affectedConcepts.pop()
        L[c].remove_object(g)  # Удаляем объект g из объема понятия L[c].Extent
        
        for parent in L.superconcepts_dict[c]:
            if parent.Affected == False:  # если верхний сосед не обработан
                affectedConcepts.append(parent)  # Добавляем верхнего соседа в список посещенных понятий
                parent.Affected = True # Помечаем верзнего соседа как посещенное понятие
                
    FastDeletion(L, minConcept)  # удаление соотв. понятий и изменения отношения покрытия

#### Формируем формальный контекст K, с которым будем работать
#### Загружаем файл test c расширением .csv

In [327]:
from fcapy.context import FormalContext
K = FormalContext.read_csv('test.csv')

##### Выводим формальный контекст

In [328]:
print(K)

FormalContext (5 objects, 5 attributes, 15 connections)
 |a|b|c|d|e|
1|X|X|X|X|X|
2|X| |X| |X|
3| | | |X| |
4| |X| | |X|
5|X|X|X|X| |


#### Формируем решетку понятий на основе формального контекста K

In [329]:
from fcapy.lattice import ConceptLattice
L = ConceptLattice.from_context(K)

In [330]:
for i in range(len(L)):
    c = L[i]
    print(i, c.extent, c.intent)

0 ('1', '2', '3', '4', '5') ()
1 ('1', '2', '4') ('e',)
2 ('1', '2', '5') ('a', 'c')
3 ('1', '3', '5') ('d',)
4 ('1', '4', '5') ('b',)
5 ('1', '2') ('a', 'c', 'e')
6 ('1', '4') ('b', 'e')
7 ('1', '5') ('a', 'b', 'c', 'd')
8 ('1',) ('a', 'b', 'c', 'd', 'e')


In [331]:
affectedConcepts = []

Для дальнейшей работы нужно реализовать следующие методы:
<ol>
<li><b>remove_object(object)</b> для класса FormalContext, который удаляет из объема формального понятия (c.extent) объект object</li>
<li><b>removeLinks(c1, c2)</b> для удаление связей между формальными понятиями c1 и c2</li>
<li><b>setLinks(c1, c2)</b> для установки связей между формальными понятиями c1 и c2</li>
</ol>